# Lebensmittel Data Visualization

In [7]:
import pandas as pd
ds_path = "data/LEBENSMITTEL.xlsx"
ds_path_others = "data/others_expenses.xlsx"
ds_path_extra = "data/extra_expenses.xlsx"
df_lebensmittel = pd.read_excel(ds_path, index_col=None)
df_others = pd.read_excel(ds_path_others, index_col=None)
df_extra = pd.read_excel(ds_path_extra, index_col=None)
df = df_lebensmittel 

In [8]:
df.head()

,product,detail,category,brand,quantity,unit,origin,angebote,laden,filiale,stadt,land,datum,preis/unit,preis,currency
0,DeutschlandTicket,NaN,Mobilität,DB,1.0,units,deutschland,0.0,Deutsche Bahn,0.0,Wiesbaden,Deutschland,2025-06-25,58.00,58.00,euro
1,Creme,Barriereschutz,Skin Care,Balea med,2.0,units,NaN,0.0,DM,NaN,Wiesbaden,Deutschland,2025-07-03,4.45,8.90,euro
2,WC Stein,Blau Kraft,Bad,WC-Frisch,1.0,units,deutschland,0.0,DM,NaN,Wiesbaden,Deutschland,2025-07-03,3.25,3.25,euro
3,KüchenPapier,3 lagig 4 stuck,Küche,Saugstark&Sicher,1.0,units,deutschland,0.0,DM,NaN,Wiesbaden,Deutschland,2025-07-03,2.85,2.85,euro
4,Wärmflasche berry,NaN,Other,NaN,1.0,units,NaN,NaN,ROSSMAN,NaN,Wiesbaden,Deutschland,2025-07-31,5.49,5.49,euro


In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 90 entries, 0 to 89
Data columns (total 16 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   product     90 non-null     object        
 1   detail      18 non-null     object        
 2   category    29 non-null     object        
 3   brand       23 non-null     object        
 4   quantity    31 non-null     float64       
 5   unit        27 non-null     object        
 6   origin      16 non-null     object        
 7   angebote    12 non-null     float64       
 8   laden       82 non-null     object        
 9   filiale     3 non-null      float64       
 10  stadt       27 non-null     object        
 11  land        27 non-null     object        
 12  datum       90 non-null     datetime64[ns]
 13  preis/unit  90 non-null     float64       
 14  preis       90 non-null     float64       
 15  currency    90 non-null     object        
dtypes: datetime64[ns](1), float6

In [10]:
from toolkit.utils import convert_bolean_columns
convert_bolean_columns(df)
df["bio?"] = df["bio?"].astype("category")
df["vegan"] = df["vegan"].astype("category")
df["origin"] = df["origin"].astype("category")
df["angebote"] = df["angebote"].astype("category")
df["laden"] = df["laden"].astype("category")
df["filiale"] = df["filiale"].astype("category")
df["currency"] = df["currency"].astype("category")
df["stadt"] = df["stadt"].astype("category")
df["land"] = df["land"].astype("category")
df["product"] = df["product"].astype("category")
df["detail"] = df["detail"].astype("string")
df["brand"] = df["brand"].astype("category")
df["unit"] = df["unit"].astype("category")
df['preis'] = df['preis'].astype(str).str.replace(',', '.').astype(float)
df['datum'] = pd.to_datetime(df['datum'], format="%d.%m.%y")
df.info()


KeyError: 'bio?'

In [ ]:
duplicated = df.duplicated().sum()
print(f"Registros duplicados en df: {duplicated}")

In [ ]:
from toolkit.utils import get_nulll_data_info
get_nulll_data_info(df)

In [ ]:
from toolkit.utils import clean_text
df['product'] = df['product'].apply(clean_text)

## Distribución de productos comprados según:

In [ ]:
from toolkit.utils import graph_pie
column_names = [
    "laden",
    "origin",
    "brand",
    "product",
    "bio?",
    "vegan",
    "category"
]
graph_pie(df, column_names=column_names, num_columns=2, figsize=(20, 25))

## Calculo de gastos a través del tiempo

In [ ]:
print(df['datum'].dtype)

In [ ]:
from toolkit.utils import plot_expense_line
plot_expense_line(df)

In [ ]:
plot_expense_line(df, group_by='product') 

## Gastos mensuales

In [ ]:
# Create Year-Month period
df['year_month'] = df['datum'].dt.to_period('M')

# Group by month and sum
monthly_sum = df.groupby('year_month')['preis'].sum().sort_index()

# Calculate cumulative sum
accumulated_expense = monthly_sum.cumsum()

# Combine into one DataFrame for display
result = pd.DataFrame({
    'Month': monthly_sum.index.strftime('%Y-%m'), 
    'Monthly Expense (€)': monthly_sum.values,
    'Cumulative Expense (€)': accumulated_expense.values
})

# Show the table
print(result)